In [1]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pypi-iarc-packages/
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp -r /kaggle/input/pypi-iarc-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

Looking in links: file:///kaggle/input/pypi-iarc-packages/
Processing /kaggle/input/pypi-iarc-packages/tabpfn-0.1.9-py3-none-any.whl


In [2]:
import numpy as np
import pandas as pd
import joblib
import optuna
import xgboost as xgb
from functools import partial

# custom modules
from preproc import load_data,scale_data
#from calibration1 import calibrate_probs,optimize_calibration
from calibration2 import calibrate_probs,optimize_calibration

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def balanced_logloss_(y_pred, y_true, eps=1e-7):
    n0 = np.sum(1-y_true)
    n1 = np.sum(y_true)
    p1 = np.clip(y_pred, eps, 1-eps)
    p0 = 1-p1
    log_loss0 = - np.sum((1-y_true) * np.log(p0)) / (n0+eps)
    log_loss1 = - np.sum(y_true * np.log(p1)) / (n1+eps)
    return (log_loss0 + log_loss1)/2


def compute_overall_metric(oof_dfs:list) -> float:
    all_metrics = [
        balanced_logloss_(oof.pred_proba.values, oof.Class.values)
        for oof in oof_dfs
    ]
    return np.mean(all_metrics)


def calibrate_oof(oof_dfs, calib_params):
    oof_dfs_calibrated = list()
    
    for oof in oof_dfs:
        oof = oof.copy(deep=True)
        calib_p1 = calibrate_probs(
            oof.pred_proba.values,
            **calib_params
        )
        oof["pred_proba"] = calib_p1
        oof_dfs_calibrated.append(oof)
        
    return oof_dfs_calibrated

In [4]:
# notebook parameter

CALIBRATE_FIRST = True

***
# load data and predictions

### 1. `lgbm-gbrt`

In [5]:
models_m01 = joblib.load("/kaggle/input/iarc-lgbm-gbrt-bagging-balanced/models.pkl")
calib_params_m01 = joblib.load("/kaggle/input/iarc-lgbm-gbrt-bagging-balanced/calib_params.pkl")
oof_dfs_m01 = joblib.load("/kaggle/input/iarc-lgbm-gbrt-bagging-balanced/oof_dataframes.pkl")

In [6]:
compute_overall_metric(oof_dfs_m01)

0.24033928714260183

In [7]:
oof_dfs_calib_m01 = calibrate_oof(oof_dfs_m01, calib_params_m01)
compute_overall_metric(oof_dfs_calib_m01)

0.22139398049564257

In [8]:
%%time
train,test,input_cols = load_data()

predicted_probs = [
    model.predict(test[input_cols]) 
    for model in models_m01
]
predicted_probs = np.mean(predicted_probs, axis=0)

if CALIBRATE_FIRST:
    predicted_probs = calibrate_probs(predicted_probs, **calib_params_m01)
predicted_probs_m01 = np.clip(predicted_probs, a_min=1e-7, a_max=1-1e-7)

del train,test

CPU times: user 224 ms, sys: 7.47 ms, total: 231 ms
Wall time: 183 ms


### 2. `lgbm-linear`

In [9]:
models_m02 = joblib.load("/kaggle/input/iarc-lgbm-linear-bagging-balanced/models.pkl")
calib_params_m02 = joblib.load("/kaggle/input/iarc-lgbm-linear-bagging-balanced/calib_params.pkl")
oof_dfs_m02 = joblib.load("/kaggle/input/iarc-lgbm-linear-bagging-balanced/oof_dataframes.pkl")

In [10]:
compute_overall_metric(oof_dfs_m02)

0.23776284612591142

In [11]:
oof_dfs_calib_m02 = calibrate_oof(oof_dfs_m02, calib_params_m02)
compute_overall_metric(oof_dfs_calib_m02)

0.21870748528950434

In [12]:
%%time
train,test,input_cols = load_data()
train,test,scaler = scale_data(train, test, input_cols)

predicted_probs = [
    model.predict(test[input_cols])
    for model in models_m02
]
predicted_probs = np.mean(predicted_probs, axis=0)

if CALIBRATE_FIRST:
    predicted_probs = calibrate_probs(predicted_probs, **calib_params_m02)
predicted_probs_m02 = np.clip(predicted_probs, a_min=1e-7, a_max=1-1e-7)

del train,test,scaler

CPU times: user 250 ms, sys: 5.39 ms, total: 256 ms
Wall time: 180 ms


### 3. `catboost`

In [13]:
models_m03 = joblib.load("/kaggle/input/iarc-catboost-weight-balanced/models.pkl")
calib_params_m03 = joblib.load("/kaggle/input/iarc-catboost-weight-balanced/calib_params.pkl")
oof_dfs_m03 = joblib.load("/kaggle/input/iarc-catboost-weight-balanced/oof_dataframes.pkl")

In [14]:
compute_overall_metric(oof_dfs_m03)

0.2633244023340282

In [15]:
oof_dfs_calib_m03 = calibrate_oof(oof_dfs_m03, calib_params_m03)
compute_overall_metric(oof_dfs_calib_m03)

0.22835158860669114

In [16]:
%%time
train,test,input_cols = load_data()

predicted_probs = [
    model.predict(test[input_cols], prediction_type="Probability")[:,1]
    for model in models_m03
]
predicted_probs = np.mean(predicted_probs, axis=0)

if CALIBRATE_FIRST:
    predicted_probs = calibrate_probs(predicted_probs, **calib_params_m03)
predicted_probs_m03 = np.clip(predicted_probs, a_min=1e-7, a_max=1-1e-7)

del train,test

CPU times: user 274 ms, sys: 18.2 ms, total: 292 ms
Wall time: 290 ms


### 4. `tabpfn`

In [17]:
models_m04 = joblib.load("/kaggle/input/iarc-tabpfn/models.pkl")
calib_params_m04 = joblib.load("/kaggle/input/iarc-tabpfn/calib_params.pkl")
oof_dfs_m04 = joblib.load("/kaggle/input/iarc-tabpfn/oof_dataframes.pkl")

In [18]:
compute_overall_metric(oof_dfs_m04)

0.3900050155836857

In [19]:
oof_dfs_calib_m04 = calibrate_oof(oof_dfs_m04, calib_params_m04)
compute_overall_metric(oof_dfs_calib_m04)

0.28839025101520327

In [20]:
%%time
train,test,input_cols = load_data()
train,test,scaler = scale_data(train, test, input_cols, impute_only=True)

predicted_probs = [
    model.predict_proba(test[input_cols])[:,1] 
    for model in models_m04
]
predicted_probs = np.mean(predicted_probs, axis=0)

if CALIBRATE_FIRST:
    predicted_probs = calibrate_probs(predicted_probs, **calib_params_m04)
predicted_probs_m04 = np.clip(predicted_probs, a_min=1e-7, a_max=1-1e-7)

del train,test,scaler

CPU times: user 2min 33s, sys: 2.5 s, total: 2min 35s
Wall time: 1min 30s


### 5. `multiout-mlp`

In [21]:
models_m05 = joblib.load("/kaggle/input/iarc-multiout-tf-mlp/models.pkl")
calib_params_m05 = joblib.load("/kaggle/input/iarc-multiout-tf-mlp/calib_params.pkl")
oof_dfs_m05 = joblib.load("/kaggle/input/iarc-multiout-tf-mlp/oof_dataframes.pkl")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [22]:
compute_overall_metric(oof_dfs_m05)

0.30563847176445524

In [23]:
oof_dfs_calib_m05 = calibrate_oof(oof_dfs_m05, calib_params_m05)
compute_overall_metric(oof_dfs_calib_m05)

0.2904142458122567

In [24]:
%%time
train,test,input_cols = load_data()
train,test,scaler = scale_data(train, test, input_cols)

predicted_probs = [
    model.predict(test[input_cols].values, verbose=0)[0][:,1:].sum(axis=1)
    for model in models_m05
]
predicted_probs = np.mean(predicted_probs, axis=0)

if CALIBRATE_FIRST:
    predicted_probs = calibrate_probs(predicted_probs, **calib_params_m05)
predicted_probs_m05 = np.clip(predicted_probs, a_min=1e-7, a_max=1-1e-7)

del train,test,scaler

CPU times: user 8.07 s, sys: 73.8 ms, total: 8.14 s
Wall time: 8.29 s


### 6. `xgb-gblinear`

In [25]:
models_m06 = joblib.load("/kaggle/input/iarc-xgb-gblinear/models.pkl")
calib_params_m06 = joblib.load("/kaggle/input/iarc-xgb-gblinear/calib_params.pkl")
oof_dfs_m06 = joblib.load("/kaggle/input/iarc-xgb-gblinear/oof_dataframes.pkl")

[20:07:09] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[20:07:09] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[20:07:09] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

In [26]:
compute_overall_metric(oof_dfs_m06)

0.37894304572238047

In [27]:
oof_dfs_calib_m06 = calibrate_oof(oof_dfs_m06, calib_params_m06)
compute_overall_metric(oof_dfs_calib_m06)

0.36502436920847003

In [28]:
%%time
train,test,input_cols = load_data()
train,test,scaler = scale_data(train, test, input_cols)

predicted_probs = [
    model.predict(xgb.DMatrix(test[input_cols]))
    for model in models_m06
]
predicted_probs = np.mean(predicted_probs, axis=0)

if CALIBRATE_FIRST:
    predicted_probs = calibrate_probs(predicted_probs, **calib_params_m06)
predicted_probs_m06 = np.clip(predicted_probs, a_min=1e-7, a_max=1-1e-7)

del train,test,scaler

CPU times: user 266 ms, sys: 6.82 ms, total: 273 ms
Wall time: 289 ms


### 7. xgb-gbtree

In [29]:
models_m07 = joblib.load("/kaggle/input/iarc-xgb-gbtree/models.pkl")
calib_params_m07 = joblib.load("/kaggle/input/iarc-xgb-gbtree/calib_params.pkl")
oof_dfs_m07 = joblib.load("/kaggle/input/iarc-xgb-gbtree/oof_dataframes.pkl")

[20:07:10] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[20:07:10] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[20:07:10] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

In [30]:
compute_overall_metric(oof_dfs_m07)

0.27239995042577647

In [31]:
oof_dfs_calib_m07 = calibrate_oof(oof_dfs_m07, calib_params_m07)
compute_overall_metric(oof_dfs_calib_m07)

0.22564184381208174

In [32]:
%%time
train,test,ianput_cols = load_data()

predicted_probs = [
    model.predict(xgb.DMatrix(test[input_cols]))
    for model in models_m07
]
predicted_probs = np.mean(predicted_probs, axis=0)

if CALIBRATE_FIRST:
    predicted_probs = calibrate_probs(predicted_probs, **calib_params_m07)
predicted_probs_m07 = np.clip(predicted_probs, a_min=1e-7, a_max=1-1e-7)

del train,test

CPU times: user 295 ms, sys: 2.98 ms, total: 298 ms
Wall time: 302 ms


***
# blend optimization

In [33]:
# models to consider

if not CALIBRATE_FIRST:
    oof_all = [
        oof_dfs_m01,
        oof_dfs_m02,
        oof_dfs_m03,
        oof_dfs_m04,
        oof_dfs_m05,
        oof_dfs_m06,
        oof_dfs_m07,
    ]
else:
    oof_all = [
        oof_dfs_calib_m01,
        oof_dfs_calib_m02,
        oof_dfs_calib_m03,
        oof_dfs_calib_m04,
        oof_dfs_calib_m05,
        oof_dfs_calib_m06,
        oof_dfs_calib_m07,
    ]

In [34]:
def compute_blend_metric(oof_all, weights):
    n_models = len(oof_all)
    n_repeats = 10
    size = len(oof_all[0][0])
   
    all_metrics = list()
    oof_dfs_blend = list()

    for repeat in range(n_repeats):
        pred_proba = np.zeros(size)
        target = oof_all[0][repeat].Class.values
        _oof = oof_all[0][repeat].copy(deep=True)

        for i in range(n_models):
            pred_proba += oof_all[i][repeat].pred_proba * weights[i]
            
        _oof["pred_proba"] = pred_proba    

        all_metrics.append(balanced_logloss_(pred_proba, target))
        oof_dfs_blend.append(_oof)

    return np.mean(all_metrics),oof_dfs_blend

def objective(oof_all, trial):
    
    n_models = len(oof_all)
    
    weights = np.array([
        trial.suggest_float(f"w_{i+1}", 0, 1)
        for i in range(n_models)
    ])
    weights /= weights.sum()
    
    return compute_blend_metric(oof_all, weights)[0]

In [35]:
optuna.logging.set_verbosity(optuna.logging.INFO)

study = optuna.create_study(
    direction='minimize',
)
study.optimize(
    partial(objective, oof_all), 
    n_trials=1000,
    n_jobs=1, 
    gc_after_trial=True,
)
print("best_value:", study.best_value)
print("best_params:", study.best_params)

[I 2023-08-09 20:07:16,076] A new study created in memory with name: no-name-c97fe4f0-4824-4a87-ad29-05506bf8a062
[I 2023-08-09 20:07:16,129] Trial 0 finished with value: 0.22258183936070736 and parameters: {'w_1': 0.08823861183857629, 'w_2': 0.16608686027521813, 'w_3': 0.38004556952415847, 'w_4': 0.7447831347335222, 'w_5': 0.1889111154250751, 'w_6': 0.2010668586009644, 'w_7': 0.26350727221939396}. Best is trial 0 with value: 0.22258183936070736.
[I 2023-08-09 20:07:16,799] Trial 1 finished with value: 0.21653187105355406 and parameters: {'w_1': 0.5415494263271464, 'w_2': 0.4130964830116669, 'w_3': 0.18933185102557015, 'w_4': 0.6289380563295494, 'w_5': 0.5352231991487688, 'w_6': 0.2935587503785839, 'w_7': 0.7534100388425827}. Best is trial 1 with value: 0.21653187105355406.
[I 2023-08-09 20:07:17,466] Trial 2 finished with value: 0.22211653832888145 and parameters: {'w_1': 0.36765478364272164, 'w_2': 0.9749603775151453, 'w_3': 0.8880950585248482, 'w_4': 0.33310139200516287, 'w_5': 0.12

best_value: 0.20722404354042548
best_params: {'w_1': 0.7734522645098655, 'w_2': 0.6389296945716858, 'w_3': 0.07282514155925465, 'w_4': 0.19356424511467327, 'w_5': 0.32503082093704916, 'w_6': 0.0002948215633821912, 'w_7': 0.02530126148559162}


In [36]:
weights = np.array([study.best_params[f"w_{i+1}"] for i in range(len(oof_all))])
weights /= weights.sum()
print(weights)

[3.81123944e-01 3.14837019e-01 3.58850913e-02 9.53801183e-02
 1.60161181e-01 1.45275361e-04 1.24673713e-02]


In [37]:
# sanity check
metric,oof_dfs_blend = compute_blend_metric(oof_all, weights)
metric

0.20722404354042548

In [38]:
# sanity check
compute_overall_metric(oof_dfs_blend)

0.20722404354042548

In [39]:
calib_params_blend = optimize_calibration(oof_dfs_blend)

[I 2023-08-09 20:20:10,804] A new study created in memory with name: no-name-2ba81b19-d4b7-4fd5-839c-9f9a8552a690
[I 2023-08-09 20:20:10,813] Trial 0 finished with value: 0.33327673711451383 and parameters: {'alpha': 8.964087479061075, 'beta': 6.400563620633348, 'gamma': 0.8321399438780032}. Best is trial 0 with value: 0.33327673711451383.
[I 2023-08-09 20:20:11,461] Trial 1 finished with value: 0.2416469302893593 and parameters: {'alpha': 3.10649385580425, 'beta': 6.531952391126952, 'gamma': 0.594834873936464}. Best is trial 1 with value: 0.2416469302893593.
[I 2023-08-09 20:20:12,110] Trial 2 finished with value: 0.22691663056540032 and parameters: {'alpha': 2.6662930677466026, 'beta': 3.7209920361193523, 'gamma': 0.5308946372954141}. Best is trial 2 with value: 0.22691663056540032.
[I 2023-08-09 20:20:12,767] Trial 3 finished with value: 0.23729680556834332 and parameters: {'alpha': 2.9918356402401174, 'beta': 5.324765499103201, 'gamma': 0.9603609881741636}. Best is trial 2 with val

best_value: 0.20278318965188197
best_params: {'alpha': 1.5378365783248964, 'beta': 1.0061763408450275, 'gamma': 0.5693489040548134}


In [40]:
# sanity check
oof_dfs_calib_blend = calibrate_oof(oof_dfs_blend, calib_params_blend)
compute_overall_metric(oof_dfs_calib_blend)

0.20278318965188197

***
# blend predictions

In [41]:
predicted_probs_all = [
    predicted_probs_m01,
    predicted_probs_m02,
    predicted_probs_m03,
    predicted_probs_m04,
    predicted_probs_m05,
    predicted_probs_m06,
    predicted_probs_m07,
]

In [42]:
predicted_probs_blend = np.zeros_like(predicted_probs_m01)
n_models = len(predicted_probs_all)

for i in range(n_models):
    predicted_probs_blend += predicted_probs_all[i]*weights[i] 

In [43]:
predicted_probs_blend = calibrate_probs(predicted_probs_blend, **calib_params_blend)
predicted_probs_blend = np.clip(predicted_probs_blend, a_min=1e-7, a_max=1-1e-7)
predicted_probs_blend

array([0.6110861, 0.6110861, 0.6110861, 0.6110861, 0.6110861])

In [44]:
sub = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")
sub["class_0"] = 1-predicted_probs_blend
sub["class_1"] = predicted_probs_blend
sub.to_csv("submission.csv", index=False)
sub.describe()

,class_0,class_1
count,5.000000,5.000000
mean,0.388914,0.611086
std,0.000000,0.000000
min,0.388914,0.611086
25%,0.388914,0.611086
50%,0.388914,0.611086
75%,0.388914,0.611086
max,0.388914,0.611086


***